## 3.3. Synthetic Regression Data 线性回归的简洁实现

在过去的几年里，出于对深度学习强烈的兴趣， 许多公司、学者和业余爱好者开发了各种成熟的开源框架。 这些框架可以自动化基于梯度的学习算法中重复性的工作。 在 3.2节中，我们只运用了：
  （1）通过张量来进行数据存储和线性代数；
  （2）通过自动微分来计算梯度。 
实际上，由于数据迭代器、损失函数、优化器和神经网络层很常用， 现代深度学习库也为我们实现了这些组件。

本节将介绍如何通过使用深度学习框架来简洁地实现 3.2节中的线性回归模型。

### 3.3.1. Generating the Dataset 生成数据集

In [10]:
from mxnet import autograd, gluon, np, npx
import tools_func as d2l

npx.set_np()

true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

### 3.3.2. 读取数据集

我们可以调用框架中现有的API来读取数据。 我们将features和labels作为API的参数传递，并通过数据迭代器指定batch_size。 此外，布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据。

In [11]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个Gluon数据迭代器"""
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

# 读取并打印第一个小批量样本
next(iter(data_iter))

[array([[ 0.41850296,  1.1516623 ],
        [-0.6084363 , -0.45579144],
        [ 0.02321943,  0.8606718 ],
        [-1.3058014 , -2.2582073 ],
        [-0.77778006,  0.12132412],
        [ 1.3584051 ,  0.5329686 ],
        [-0.3085847 ,  0.29681763],
        [ 1.4356098 ,  0.46452948],
        [-1.5964411 ,  0.1162167 ],
        [ 0.59155923, -0.3203392 ]]),
 array([[1.1012753 ],
        [4.5438824 ],
        [1.3045392 ],
        [9.260045  ],
        [2.2251105 ],
        [5.1088567 ],
        [2.5737898 ],
        [5.4797077 ],
        [0.62877166],
        [6.4719415 ]])]

### 3.3.3. Defining the Model 定义模型

当我们在 3.2节中实现线性回归时， 我们明确定义了模型参数变量，并编写了计算的代码，这样通过基本的线性代数运算得到输出。 但是，如果模型变得更加复杂，且当我们几乎每天都需要实现模型时，自然会想简化这个过程。 这种情况类似于为自己的博客从零开始编写网页。 做一两次是有益的，但如果每个新博客就需要工程师花一个月的时间重新开始编写网页，那并不高效。

对于标准深度学习模型，我们可以使用框架的预定义好的层。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。 我们首先定义一个模型变量net，它是一个Sequential类的实例。 Sequential类将多个层串联在一起。 当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。 在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。 但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉“标准的流水线”。

![](/Users/chenjunming/Desktop/FinTech/DeepLearning/pics/WX20240701-103804@2x.png)
回顾 上图中的单层网络架构， 这一单层被称为全连接层（fully-connected layer）， 因为它的每一个输入都通过矩阵-向量乘法得到它的每个输出。

在mxnet的Gluon中，全连接层在Dense类中定义。 由于我们只想得到一个标量输出，所以我们将该数字设置为1。值得注意的是，为了方便使用，Gluon并不要求我们为每个层指定输入的形状。 所以在这里，我们不需要告诉Gluon有多少输入进入这一层。 当我们第一次尝试通过我们的模型传递数据时，例如，当后面执行net(X)时， Gluon会自动推断每个层输入的形状。 本节稍后将详细介绍这种工作机制。

在PyTorch中，全连接层在Linear类中定义。 值得注意的是，我们将两个参数传递到nn.Linear中。 第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。

下面使用mxnet的gluon来定义模型：

In [12]:
# nn是神经网络的缩写
from mxnet.gluon import nn

net = nn.Sequential()
net.add(nn.Dense(1))

### 3.3.4. Initializing Model Parameters 初始化模型参数

在使用net之前，我们需要初始化模型参数。如在线性回归模型中的权重和偏置。深度学习框架通常有预定义的方法来初始化参数。 在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样， 偏置参数将初始化为零。

我们从MXNet导入initializer模块，这个模块提供了各种模型参数初始化方法。 Gluon将init作为访问initializer包的快捷方式。 我们可以通过调用init.Normal(sigma=0.01)来指定初始化权重的方法。 默认情况下，偏置参数初始化为零。

In [13]:
from mxnet import init

net.initialize(init.Normal(sigma=0.01))

上面的代码可能看起来很简单，但是这里有一个应该注意到的细节： 我们正在为网络初始化参数，而Gluon还不知道输入将有多少维! 网络的输入可能有2维，也可能有2000维。 Gluon让我们避免了这个问题，在后端执行时，初始化实际上是推迟（deferred）执行的， 只有在我们第一次尝试通过网络传递数据时才会进行真正的初始化。 请注意，因为参数还没有初始化，所以我们不能访问或操作它们。

### 3.3.5. Defining the Loss Function 定义损失函数

在Gluon中，loss模块定义了各种损失函数。 在这个例子中，我们将使用Gluon中的均方误差（L2Loss）。

In [14]:
loss = gluon.loss.L2Loss()

### 3.3.6. Defining the Optimization Algorithm 定义优化算法

小批量随机梯度下降算法是一种优化神经网络的标准工具， Gluon通过Trainer类支持该算法的许多变种。 当我们实例化Trainer时，我们要指定优化的参数 （可通过net.collect_params()从我们的模型net中获得）、 我们希望使用的优化算法（sgd）以及优化算法所需的超参数字典。 小批量随机梯度下降只需要设置learning_rate值，这里设置为0.03。

In [15]:
from mxnet import gluon

'''
该函数用于初始化一个训练器（trainer），它使用小批量随机梯度下降（SGD）作为优化算法，以学习率为0.03来训练网络参数。
其中，net.collect_params()用于获取网络的所有参数，'sgd'指定了优化算法类型，而{'learning_rate': 0.03}是一个字典，用于设置SGD优化算法的超参数。
'''
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

### 3.3.7. Training 训练

通过深度学习框架的高级API来实现我们的模型只需要相对较少的代码。 我们不必单独分配参数、不必定义我们的损失函数，也不必手动实现小批量随机梯度下降。 当我们需要更复杂的模型时，高级API的优势将大大增加。 当我们有了所有的基本组件，训练过程代码与我们从零开始实现时所做的非常相似。

回顾一下：在每个迭代周期里，我们将完整遍历一次数据集（train_data）， 不停地从中获取一个小批量的输入和相应的标签。 对于每一个小批量，我们会进行以下步骤:

通过调用net(X)生成预测并计算损失l（前向传播）。

通过进行反向传播来计算梯度。

通过调用优化器来更新模型参数。

为了更好的衡量训练效果，我们计算每个迭代周期后的损失，并打印它来监控训练过程。

In [17]:
'''
这段代码是一个训练神经网络的循环。简单来说，它在每个epoch（轮次）上遍历数据集，并对每个batch进行以下操作：
使用autograd.record()记录计算过程，以便之后进行反向传播。
1. 计算当前batch的损失函数值l。
2. 调用l.backward()进行反向传播，计算梯度。
3. 使用trainer.step(batch_size)更新模型参数，即进行一次优化器的迭代步。 
4. 在每个epoch结束后，还会计算整个数据集的损失函数值，并打印出当前epoch和损失值。
'''
num_epochs = 3  # 设定了模型训练的总轮次（epochs）
for epoch in range(num_epochs):
    for X, y in data_iter:  # data_iter是一个数据迭代器，每次迭代会返回一个数据样本X和对应的标签y。
        with autograd.record(): # 创建了一个自动记录执行过程的上下文, 即计算图; 在这个上下文中，执行的操作都会被记录下来，以便后续进行反向传播计算梯度。
            l = loss(net(X), y)
        l.backward()    # 用于反向传播的函数，用于计算损失函数关于模型参数的梯度。调用该函数后，系统会从输出端开始，按照计算图的反向顺序计算梯度，并将梯度存储在模型参数的.grad属性中。
        trainer.step(batch_size)    # 执行一次训练迭代，利用SGD算法根据batch_size大小的当前批次数据计算出的梯度来更新模型参数。
    l = loss(net(features), labels) # 计算当前epoch的损失函数值
    print(f'epoch {epoch + 1}, loss {l.mean().asnumpy():f}')

epoch 1, loss 0.000049
epoch 2, loss 0.000049
epoch 3, loss 0.000049


In [20]:
# 下面我们比较生成数据集的真实参数和通过有限数据训练获得的模型参数。 
# 要访问参数，我们首先从net访问所需的层，然后读取该层的权重和偏置。 正如在从零开始实现中一样，我们估计得到的参数与生成数据的真实参数非常接近。
w = net[0].weight.data()
print(w)
print(f'w的估计误差： {true_w - w.reshape(true_w.shape)}')
b = net[0].bias.data()
print(b)
print(f'b的估计误差： {true_b - b}')

[[ 2.0006518 -3.4001272]]
w的估计误差： [-0.00065184  0.00012708]
[4.1995068]
b的估计误差： [0.00049305]


### 小结

我们可以使用Gluon更简洁地实现模型。

在Gluon中，data模块提供了数据处理工具，nn模块定义了大量的神经网络层，loss模块定义了许多常见的损失函数。

MXNet的initializer模块提供了各种模型参数初始化方法。

维度和存储可以自动推断，但注意不要在初始化参数之前尝试访问参数。